# <center>Scrapy para usuários de bs4</center>



Henrique Coura

## about:me

- Engenheiro Mecânico
- Programo há 3 anos e meio, 2 anos profissionalmente
- Software Engineer na Scrapinghub
- Apaixonado por dados: scraping, data processing, machine learning, etc.
- https://github.com/hcoura
- https://medium.com/@henriquecoura_87435
- coura.henrique@gmail.com


## bs4 vs scrapy

- É como comparar maçãs e laranjas
- scrapy é um framework para scraping
- Beautiful Soup é um lib para extração de dados de arquivos HTML e XML. Funciona com vários `parsers` e providencia formas idiomáticas de busca, navegação e modificação da árvore de parsing. (Tradução da descrição deles mesmo)
- Se quiser, não que eu recomende, pode até usar o bs4 junto com scrapy
- bs4 é bom para scripts mais curtos, que só vão rodar algumas vezes
- Scrapy é bom para projetos maiores que envolvem maiores complexidades e precisam de uma estrutura mais robusta

## libs semelhantes ao bs4

Existem outras libs que você pode utilizar para o mesmo caso de uso do bs4, essas são algumas:

- lxml - muito utilizada como base para outras libs, suporta vários parsers(inclusive o do próprio bs4). Encontra elementos via XPATH
- parsel - um wrapper para o lxml utilizado pelo scrapy para parsing. Encontra elementos via XPATH e CSS SELECTORS
- requests-html - lib novinha que junta um monte de coisa para scraping básico(requests, XPATH, CSS SELECTORS, Mocked user-agent, seguir redirects, persistencia de cookies, renderização de JS, etc.)

https://github.com/lxml/lxml

https://github.com/kennethreitz/requests-html

https://github.com/scrapy/parsel

## Comparando bs4 e parsel

- Eu não acho a api do bs4 muito intuitiva, ainda mais considerando que é diferente de tudo mais na indústria ao contrário de xpath / css selectors

- Por outro lado algumas pessoas devem achar a api do bs4 mais fácil de trabalhar

- O parser do bs4 lida um pouco melhor com HTML quebrado mas é mais lento (é possível usar o parser do bs4 no lxml e vice versa)

- bs4 não suporta XPATH

- parsel usa o lxml e adiciona CSS SELECTORS além de fornecer uma interface um pouco mais fácil de lidar

- o scrapy usa o parsel para fazer o parsing por padrão

In [2]:
import re
import requests
from urllib.parse import unquote

from bs4 import BeautifulSoup
from parsel import Selector

# <center>bs4</center>

In [3]:
url = 'http://localhost:8000/wikipedia/Clube_Atlético_Mineiro.html'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [3]:
divs = soup.find_all('div', {'class': 'mw-parser-output'})
image = None
for div in divs:
    image = div.find('table').find('img')
    if image:
        break
image['src']

'images/120px-Atletico_mineiro_galo.png'

In [4]:
title = soup.find(id='firstHeading').text
title

'Clube Atlético Mineiro'

In [5]:
related_links = soup.find('div', id='bodyContent').find_all('a', href=re.compile('^[^:]*\.html$'))
related_links = [unquote(link['href']) for link in related_links]
related_links[:5]

['Galo.html',
 '25_de_março.html',
 '1908.html',
 'Estádio_Raimundo_Sampaio.html',
 'Mineirão.html']

In [6]:
p = divs[0].find('p')
paragraph = ''
for e in p.children:
    try:
        if e.find('a', href=re.compile('#cite')):
            continue
    except:
        pass
    paragraph += e.string
paragraph

'O Clube Atlético Mineiro (conhecido apenas por Atlético e cujo acrônimo é CAM) é um clube brasileiro de futebol sediado na cidade de Belo Horizonte, Minas Gerais. Fundado em 25 de março de 1908 por um grupo de estudantes, tem como suas cores tradicionais o preto e o branco. Contudo, o clube teve como primeiro nome Athlético Mineiro Football Club . Seu símbolo e alcunha mais popular é o Galo, mascote oficial no final da década de 1930. O Atlético é um dos clubes mais populares do Brasil.'

## <center>parsel</center>

In [7]:
url = 'http://localhost:8000/wikipedia/Clube_Atlético_Mineiro.html'

response = requests.get(url)
sel = Selector(response.text)

In [8]:
image = sel.css('.mw-parser-output table img::attr(src)').extract_first()
image

'images/120px-Atletico_mineiro_galo.png'

In [9]:
title = sel.css('#firstHeading::text').extract_first()
title

'Clube Atlético Mineiro'

In [10]:
related_links = sel.xpath('//div[@id="bodyContent"]/descendant::a[re:test(@href, '
                          '"^[^:]*\.html$")]/@href').extract()
related_links = [unquote(url) for url in related_links]
related_links[:5]

['Galo.html',
 '25_de_março.html',
 '1908.html',
 'Estádio_Raimundo_Sampaio.html',
 'Mineirão.html']

In [11]:
paragraph = ''.join(sel.xpath('//div[@class="mw-parser-output"]/p[1]/descendant-or-self::*['
                                'self::a[not(starts-with(@href, "#cite"))] or self::b or self::p]/text()').extract())
paragraph

'O Clube Atlético Mineiro (conhecido apenas por Atlético e cujo acrônimo é CAM) é um clube brasileiro de futebol sediado na cidade de Belo Horizonte, Minas Gerais. Fundado em 25 de março de 1908 por um grupo de estudantes, tem como suas cores tradicionais o preto e o branco. Contudo, o clube teve como primeiro nome  . Seu símbolo e alcunha mais popular é o Galo, mascote oficial no final da década de 1930. O Atlético é um dos clubes mais populares do Brasil.'

## scrapy

**Um framework open source e colaborativo para extração de dados de websites**

- Projetos de scraping tendem a ser complexos e cheios de "erros" e "bugs" que não controlamos
- Também são cheios de "workarounds" para sites e situações específicas
- Mas mesmo assim tem muita coisa "reaproveitável" para cada caso
- As vezes é preciso "esquivar" das defesas do site
- O scrapy ajuda em tudo isso e mais um pouco além de fornecer uma estrutura para manter uma base de código mais organizada e mais "sã"

https://scrapy.org/

https://github.com/scrapy/scrapy


## Um spider simples

Pegando o mesmo exemplo da wikipédia vou mostrar como escrever um crawler que:

- Extraia uma página e suas página correlatas na wikipédia;

- De cada página extraída teremos: Título, primeiro parágrafo, url da imagem principal, imagem principal(arquivo);

- Exportar tudo para um .csv ou .json;

- Utilize várias features do scrapy para facilitar nossa vida.


```
scrapy startproject <project_name> [project_dir]
```

```
scrapy startproject scrapy_project
```

## Items


- O maior objetivo no scraping é transformar dados não-estruturados em estruturados
- Os `Items` nos dão estrutura que não temos nos dicts
- Não permitem o `assignment` em campos não declarados
- Possui uma API similar aos dicts
- Vários componentes do scrapy utilizam informações passadas pelos `Items`

https://docs.scrapy.org/en/latest/topics/items.html

## Item Loaders


- Fornecem um mecanismo conveniente para popular `Items`
- Tornam o processo de limpeza e processamento de dados durante a extração muito mais limpo e fácil de manter e fora do `Spider`

https://docs.scrapy.org/en/latest/topics/loaders.html

## Item Pipelines

- Depois que um `Item` é extraído ele é enviado aos `Item Pipelines` que os processam sequencialmente
- Cada `Item Pipeline` processam o `Item` executando alguma ação e decidindo se deve ser "jogado fora" ou continuar pelo `pipeline`
- Alguns casos de uso:
    - Limpar dados HTML
    - Validação de dados
    - Verificação de duplicados
    - Armazenar `Items`

https://docs.scrapy.org/en/latest/topics/item-pipeline.html

## Exportar para csv / json

csv
```
scrapy crawl [spider name] -o filename.csv
```

json
```
scrapy crawl [spider name] -o filename.json
```

no nosso caso
```
scrapy crawl wiki -o articles.csv
```

## Funcionalidades avançadas


### Arquitetura

![scrapy architecture](scrapy_architecture.png "Arquitetura do scrapy")

https://docs.scrapy.org/en/latest/topics/architecture.html




O fluxo de dados é o seguinte:

- (1) O `engine` pega as requisições iniciais do `spider`;
- (2) Envia as requisições para o `scheduler`;
- (3) Recebe as próximas requisições para "crawlear";
- (4) Enviar as requisições para o `downloader`, passando por todos os `downloader middlewares`;
- (5) Depois que o `downloader` termina de baixar a página, ele cria uma resposta e passa ela de volta pelos `downloader middlewares`;
- (6) O `engine`, envia as respostas para o `spider` passando pelos `spider middlewares`;
- (7) O `spider` processa a resposta e envia para o `engine` itens e requisições;
- (8) As requisições enviadas pelo spider vão para o `scheduler` e reinicia o processo, os itens são enviados para os `item pipelines` para o processamento final;
- O processo continua até não haver mais requisições no `scheduler`

<img src="hiring.png" width="80%">


# <center>Obrigado!</center>



### <center>Perguntas?</center>

### Downloader Middleware

- O `downloader middleware` é um sistema leve, de mais baixo nível, para conectar ao processo de requisições e resposta do scrapy

- Possui dois métodos principais, `process_request(request, spider)` e `process_response(request, response, spider)`

- Exemplo built-in: RetryMiddleware que controla quando tentar de novo alguma requisição

https://docs.scrapy.org/en/latest/topics/downloader-middleware.html

### Spider Middleware

- O `spider middleware` possui hooks para processar as respostas que são enviadas aos `spiders` e as requisições e items gerados pelos `spiders`

- Possui os seguintes métodos principais, `process_spider_input(response, spider)` e `process_spider_output(response, result, spider)`, `process_spider_exception(response, exception, spider)`, `process_start_requests(start_requests, spider)`

- Exemplo built-in: DepthMiddleware que controla quão profundo seu spider vai no site

https://docs.scrapy.org/en/latest/topics/spider-middleware.html

### Extensions

- Classes normais que são inicializadas no startup do scrapy, geralmente utilizadas para customizações além do que um middleware to oferece

https://docs.scrapy.org/en/latest/topics/extensions.html

### Signals

- O scrapy envia vários "sinais" para notificar quando certos eventos ocorrem, você pode pegar esses sinais para adicionar funcionalidades customizadas. Geralmente em uma `extension`, mas podem ser utilizados em outros lugares, como no próprio `spider`

https://docs.scrapy.org/en/latest/topics/signals.html